In [58]:
import requests
from bs4 import BeautifulSoup
# Use requests_html
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession

# Define the base URL and Query Parameter of the website
BASE_URL = 'https://www2.nhk.or.jp/gogaku/gendaieigo/detail/index.html'
QUERY_PARAM = '?no='

def create_url_from_date_query_param(base_url, query_param):
    # Input date and create the url to access the website for the date
    date = input("Input date:")
    url = base_url + query_param + date
    return url

def getHTMLcontents(url):
    # Get the HTML file from the URL
    html = requests.get(url)
    # Read the HTML file using BeautifulSoup
    content = BeautifulSoup(html.content, "html.parser")
    return content

# Print the created url of the website
url = create_url_from_date_query_param(BASE_URL,QUERY_PARAM)

In [59]:
print(url)

https://www2.nhk.or.jp/gogaku/gendaieigo/detail/index.html?no=20230208


In [60]:
# Use requests-html
# https://gammasoft.jp/blog/how-to-download-web-page-created-javascript/
# https://pypi.org/project/requests-html/

asession = AsyncHTMLSession()

# exec JavaScript using arender()
# https://blog.ikedaosushi.com/entry/2019/09/15/162445
async def exec_js(url):
    r = await asession.get(url)
    # Generate HTML using a browser engine (Chronium Browser)
    await r.html.arender()
    return r

In [61]:
# Neet to solve the error happend when executing "asyncio" in JupyterNotebook 
import nest_asyncio
nest_asyncio.apply()

# Execute the function "exec_js"
import asyncio
loop = asyncio.get_event_loop()
r = loop.run_until_complete(exec_js(url))

In [62]:

# CSS Selector for the news title
# wp > div.gendai-hd2 > div > div > p.gendai-hd2-info--cat
# sel = '#wp > div.gendai-hd2 > div > div > p.gendai-hd2-info--cat'
sel = '#wp > div.gendai-hd2 > div > div > p.gendai-hd2-info--broadcast'
all_elements = r.html
# element = r.html.find(sel, first=True)
print(dir(all_elements))

['__aiter__', '__anext__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_async_render', '_encoding', '_html', '_lxml', '_make_absolute', '_pq', 'absolute_links', 'add_next_symbol', 'arender', 'base_url', 'browser', 'default_encoding', 'element', 'encoding', 'find', 'full_text', 'html', 'links', 'lxml', 'next', 'next_symbol', 'page', 'pq', 'raw_html', 'render', 'search', 'search_all', 'session', 'skip_anchors', 'text', 'url', 'xpath']


In [ ]:
print(all_elements.find(sel, first=True).text)

In [64]:
elements = all_elements.find(sel, first = True).text
lines = {}
for i, line in enumerate(elements.split("\n")):
    if i < 4:
        lines[i] = line
        print('{}:{}'.format(i,lines[i]))
        print('---'*10)

0:2023年2月8日(水)の放送内容
------------------------------
1:越冬キャベツ 掘り出し作業盛ん
------------------------------
2:HOKKAIDO TOWN BEGINS SHIPPING WINTER CABBAGES
------------------------------
3:2023年1月19日のニュース
------------------------------


In [97]:
# Pickup Japan datetime format from string
import re
def pickup_datetime(text):
    s = text
    pattern = r'\d{4}年\d{1,2}月\d{1,2}日'
    return re.findall(pattern, s)

print('before:{}'.format(lines[0]))
str_date = pickup_datetime(lines[0])[0]
print('after:{}'.format(str_date))

before:2023年2月8日(水)の放送内容
after:2023年2月8日


In [66]:
# Convert the strings of Japan datetime format to the datetime object using strptime method
# https://atmarkit.itmedia.co.jp/ait/articles/2111/24/news019.html

from datetime import datetime
# str_date = '2021年11月24日'

def convert_datetime_from_jpn(text): 
    date_obj = datetime.strptime(text, '%Y年%m月%d日')
    return date_obj

air_date = convert_datetime_from_jpn(str_date).date()
print(air_date)

2023-02-08


In [67]:
print(lines)

{0: '2023年2月8日(水)の放送内容', 1: '越冬キャベツ 掘り出し作業盛ん', 2: 'HOKKAIDO TOWN BEGINS SHIPPING WINTER CABBAGES', 3: '2023年1月19日のニュース'}


In [68]:
news_date = convert_datetime_from_jpn(pickup_datetime(lines[3])[0]).date()
print(news_date)

2023-01-19


In [69]:
# replace space to underscore
def replace_space_to_underscore(text):
    words = text.split()
    line = ''
    for word in words:
        if line:
            line += word +"_"
        else:
            line += "_" + word
    return line

title = replace_space_to_underscore(lines[2])
print(title)

_HOKKAIDOTOWN_BEGINS_SHIPPING_WINTER_CABBAGES_


In [70]:
filename = str(air_date) + title + str(news_date)
print(filename)

2023-02-08_HOKKAIDOTOWN_BEGINS_SHIPPING_WINTER_CABBAGES_2023-01-19


In [73]:
movie_sel = "#wp > div.gendai-hd2 > div > div.gendai-hd2-video > div > iframe"
movie_xpath = "/html/body/div[3]/div[4]/div/div[2]/div/iframe"
print(all_elements.find(movie_sel, first=True))
# print(all_elements.xpath(movie_xpath))

<Element 'iframe' id='video--detail__media' src='./video.html?no=20230208&file=22-3508-193&thumb=22-3508-193.jpg' frameborder='0' allowfullscreen='allowfullscreen'>


In [74]:
print(dir(all_elements.find(movie_sel, first=True)))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_attrs', '_encoding', '_html', '_lxml', '_make_absolute', '_pq', 'absolute_links', 'attrs', 'base_url', 'default_encoding', 'element', 'encoding', 'find', 'full_text', 'html', 'lineno', 'links', 'lxml', 'pq', 'raw_html', 'search', 'search_all', 'session', 'skip_anchors', 'tag', 'text', 'url', 'xpath']


In [90]:
print(all_elements.find(movie_sel,first=True).url)
print(all_elements.find(movie_sel, first=True).base_url)

https://www2.nhk.or.jp/gogaku/gendaieigo/detail/index.html?no=20230208
https://www2.nhk.or.jp/gogaku/gendaieigo/detail/?no=20230208


In [92]:
print(type(all_elements.links))
print(list(all_elements.links))

<class 'set'>
['http://k.nhk.jp/', 'https://www.nhk.or.jp/info/?cid=dchk-commonheader', '/gogaku/', 'https://www6.nhk.or.jp/nhkpr/', 'https://hh.pid.nhk.or.jp/pidh10/eventList.do', 'https://www.nhk.jp/g/anime/', 'https://www.nhk-ondemand.jp/', '../', '/gogaku/french/', 'http://www.nhk-ep.com/', 'https://www.nhk.or.jp/heart-net/', 'https://www.nhk.or.jp/school/eigo/beat/', 'https://www3.nhk.or.jp/nhkworld/', 'https://www3.nhk.or.jp/news/', 'https://www.nhk.or.jp/privacy/policy/', 'https://www.nhk.or.jp/bs4k8k/', 'http://www.nhk-sc.or.jp/gogaku/', 'https://www.nhk.jp/g/drama/', 'https://www.nhk.or.jp/', '/gogaku/programs/', 'https://www.nhk.or.jp/school/', '/gogaku/#ct', '/gogaku/english/', 'https://www.nhk.or.jp/kenko/', 'https://www2.nhk.or.jp/gogaku/', 'https://www.nhk.or.jp/toppage/nhk_info/copyright.html', 'https://www3.nhk.or.jp/sports/', '/gogaku/chinese/', 'http://www.nhk-sc.or.jp/gogaku/dls/', 'https://pid.nhk.or.jp/cas/', 'https://www.nhk.or.jp/privacy/notice/general/', 'https: